In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install geffnet
!pip install git+https://github.com/pabloppp/pytorch-tools -U
!pip install thop 

In [ ]:
import torch
from geffnet import efficientnet_b1
import torch.nn as nn

In [ ]:

class CrossEntropyLabelSmoothLoss(nn.Module):
    """Cross entropy loss with label smoothing regularizer.
    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.
    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """

    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmoothLoss, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """

        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss
    

class Flat(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, epochs, anneal_start=0.65, last_epoch=-1):
        self.epochs = epochs
        self.start = anneal_start
        super(Flat, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.epochs * self.start:
            return [base_lr for base_lr in self.base_lrs]
        else:
            return [
                base_lr * (1 + math.cos(math.pi * self.last_epoch / 5)) / 2
                for base_lr in self.base_lrs
            ]

In [ ]:
from fastai.vision.all import get_image_files
import random
from sklearn.model_selection import KFold,StratifiedKFold
data=pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
image_label_dict={}
for i in range(data.shape[0]):
    info=data.iloc[i]
    image_name=info['img']
    label=info['classname']
    image_label_dict[image_name]=label
from fastai.vision.all import get_image_files
import random
from sklearn.model_selection import KFold,StratifiedKFold
names=get_image_files('/kaggle/input/state-farm-distracted-driver-detection/imgs/train')
random.shuffle(names)
X=[]
Y=[]
for i in range(len(names)):
    name=str(names[i])
    name=name.split('/')[-1]
    label=image_label_dict[name]
    X.append([i])
    Y.append (label)
    
sfolder = StratifiedKFold(n_splits=5,random_state=0,shuffle=True)

train,test=next(sfolder.split(X,Y))
train_name=[]
test_name=[]
train_label=[]
test_label=[]
for i in train:
    index=X[i][0]
    train_name.append(names[index])
    train_label.append(Y[i])
for i in test:
    test_name.append(names[index])
    test_label.append(Y[i])
import torch.nn as nn
import torch.utils.data as Data
from PIL import Image
import torchvision.transforms as T
import torch.nn.functional as F
train_transform=T.Compose([
       T.Resize([384,384]),
       T.RandomHorizontalFlip(),
       T.Pad(10),
       T.RandomCrop([384,384]),
       T.ColorJitter(),
       T.RandomAffine(degrees=10),
       T.ToTensor(),
       T.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
]
)
test_transform=T.Compose([
    T.Resize([384,384]),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])


class Dataset(Data.Dataset):
    def __init__(self,name,label,transform):
        super(Dataset, self).__init__()
        self.name=name
        self.label=label
        self.transform=transform

    def __getitem__(self, index):
        path=str(self.name[index])
        label=self.label[index]
        label=int(label.strip()[-1])
        image=Image.open(path)
        image=self.transform(image)
        return image,label

    def __len__(self):
        return len(self.name)

In [ ]:
from fastai.metrics import accuracy

class Checkpoint:
    def __init__(self, ckpt):
        self.ckpt = ckpt
        self.init = -1000

    def __call__(self, metric, model, optimizer):
        if metric > self.init:
            self.init = metric
            torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict()}, self.ckpt)

def do_eval(model,valid_dl):
    device=torch.device('cuda:0')
    accs=0
    with torch.no_grad():
        for images,labels in valid_dl:
            images=images.to(device)
            labels=labels.to(device)
            logit=model(images)
            acc=accuracy(logit,labels)
            accs+=acc
    return accs/len(valid_dl)
from torchtools.optim import RangerLars
from torch.cuda.amp import autocast,GradScaler
scaler=GradScaler()
epochs=50
train_dataset=Dataset(train_name,train_label,train_transform)
val_dataset=Dataset(test_name,test_label,test_transform)
train_dl=Data.DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=8)
val_dl=Data.DataLoader(val_dataset,batch_size=64,shuffle=False,num_workers=8)
model=efficientnet_b1(True)
model.classifier=nn.Linear(1280,10)
optimizer=RangerLars(model.parameters(),lr=0.001)
scheduler=Flat(optimizer,50)
cls_criterion=CrossEntropyLabelSmoothLoss(10)


ckpt='best.pt'
checkpoint=Checkpoint(ckpt)
scaler=torch.cuda.amp.GradScaler()
device=torch.device('cuda:0')
model=model.to(device)

from tqdm import tqdm
with tqdm(total=epochs) as pbar:
    for epoch in range(epochs):
        model.train()
        for images,labels in train_dl:
            
            images=images.to(device)
            labels=labels.to(device)
            optimizer.zero_grad()
            loss=0
            with autocast():
                logit=model(images)
                loss=loss+cls_criterion(logit,labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        scheduler.step()    
            
        if epoch%5==0:
            model.eval()
            acc=do_eval(model,val_dl)
            print('epoch:{}--acc:{}'.format(epoch,acc))
            checkpoint(acc,model,optimizer)
        pbar.update(1)